In [1]:
import sys
sys.path.append('../')

from autogpt.config import Config
import json
import requests
import yaml

from duckduckgo_search import ddg
from bs4 import BeautifulSoup


CFG = Config()

In [2]:
import re
import copy
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options as ChromeOptions

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
from sys import platform

import time
from bs4 import BeautifulSoup
import requests

## Recommend Travel Itinerary

In [3]:
from autogpt.commands.naver_travel import load_driver
from autogpt.commands.naver_travel import latlng2kor_address
from autogpt.commands.naver_travel import kor_address2latlng
from autogpt.commands.naver_travel import get_kor_weather_info
from autogpt.commands.google_search import google_search_place

from autogpt.commands.naver_travel import get_place_header
from autogpt.commands.naver_travel import get_place_main_info
from autogpt.commands.naver_travel import get_place_reviews

from autogpt.commands.naver_travel import kor_tourism_places_nearby_latlng

In [4]:
result = google_search_place('파도살롱')

In [5]:
json.loads(result)

[{'name': '파도살롱',
  'address_components': ['Gyeonggang-ro', 'Gangneung-si', 'Gangwon-do'],
  'rating': 4.3,
  'types': ['real_estate_agency', 'point_of_interest', 'establishment'],
  'location': {'lat': 37.74995759999999, 'lng': 128.8906032}}]

In [6]:
location = json.loads(result)[0]['location']
latitude, longitude = location['lat'], location['lng']

In [7]:
kor_tourism_places_nearby_latlng(latitude, longitude, query='')

TimeoutException: Message: 
Stacktrace:
#0 0x556215df1e43 <unknown>
#1 0x556215b29908 <unknown>
#2 0x556215b67d2d <unknown>
#3 0x556215b67e41 <unknown>
#4 0x556215ba8184 <unknown>
#5 0x556215b8d9dd <unknown>
#6 0x556215ba562a <unknown>
#7 0x556215b8d773 <unknown>
#8 0x556215b5a6de <unknown>
#9 0x556215b5ba72 <unknown>
#10 0x556215dc27d2 <unknown>
#11 0x556215dc4f7b <unknown>
#12 0x556215dc49f6 <unknown>
#13 0x556215dc56f5 <unknown>
#14 0x556215dcb47b <unknown>
#15 0x556215dc5aa6 <unknown>
#16 0x556215da3074 <unknown>
#17 0x556215ddeb75 <unknown>
#18 0x556215dded1e <unknown>
#19 0x556215dec56f <unknown>
#20 0x7fe9f2449ea7 start_thread


In [8]:
weather = get_kor_weather_info(latitude=37.5665, longitude=126.9780)
json.loads(weather)

{'current_temperature': '19',
 'current_status': '흐림',
 'current_wind': '남동1m/s',
 'weekly': [{'day': '5.26.금', 'status': '흐림', 'temperature': ' 16°/27° '},
  {'day': '5.27.토', 'status': '흐림', 'temperature': ' 18°/24° '},
  {'day': '5.28.일', 'status': '흐리고 비', 'temperature': ' 18°/23° '},
  {'day': '5.29.월', 'status': '흐리고 비', 'temperature': ' 18°/27° '},
  {'day': '5.30.화', 'status': '구름많음', 'temperature': ' 18°/27° '}]}

In [5]:
address_error = "Gyeonggang-ro, Gangneung-si, Gangwon-do"
address_error = "강원도 강릉시 경강로 파도살롱"

In [6]:
tourism_places = kor_tourism_places_nearby_latlng(address_error)

NameError: name 'recommend_nearby_tourism_places' is not defined

In [42]:
driver = load_driver()
driver.get("https://map.naver.com/v5/search")

# wait until the page is loaded
driver.implicitly_wait(3)

# search for the query and click enter
search_box = driver.find_element_by_css_selector("div.input_box>input.input_search")
search_box.send_keys(json.loads(address)['address_jibun'])
search_box.send_keys(Keys.ENTER)

/tmp/ipykernel_18987/1855728953.py:8: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search_box = driver.find_element_by_css_selector("div.input_box>input.input_search")


In [25]:
types = {
    '음식점': 'DINING',
    '카페': 'CAFE',
    '쇼핑': 'SHOPPING',
    '숙박': 'ACCOMMODATION',
    '병원의료': 'HOSPITAL',
    '은행': 'BANK',
    '주유소': 'OIL',
    '마트슈퍼': 'MART',
    '편의점': 'STORE',
    '생활편의': 'CONVENIENCE',
    '명소': 'SIGHTS',
    '체육시설': 'SPORTS',
    '영화공연': 'CINEMA',
    '관광서': 'GOVERNMENT',
}
type = types['음식점']

lat, lng = round(latitude, 7), round(longitude, 7)
coord = f'{lng};{lat}'

In [42]:
naver_nearby = f"https://m.map.naver.com/search2/interestSpot.naver?type={type}&searchCoord={coord}"

In [47]:
naver_nearby

'https://m.map.naver.com/search2/interestSpot.naver?type=DINING&searchCoord=128.8906032;37.7499576'

In [55]:
naver_map_search_url = f'https://m.map.naver.com/search2/search.naver?query=맛집'


In [ ]:

driver.get(naver_map_search_url)
time.sleep(2)
elements = driver.find_elements_by_css_selector('li._item._lazyImgContainer')

In [74]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
html = requests.get(naver_nearby, headers=headers).text

In [77]:
soup = BeautifulSoup(html, 'html.parser')

In [79]:
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="website" property="og:type"/>
<meta content="네이버 지도" property="og:site_name"/>
<meta content="네이버 지도" property="og:title"/>
<meta content="https://ssl.pstatic.net/static/maps/m/og_map.png" property="og:image"/>
<meta content="https://ssl.pstatic.net/static/maps/m/og_map.png" property="twitter:image"/>
<meta content="자동차/대중교통/자전거/도보 길찾기, 실시간 교통 및 버스위치, CCTV, 지하철, 버스노선, 거리뷰, 뮤지엄뷰 제공." property="og:description"/>
<meta content="자동차/대중교통/자전거/도보 길찾기, 실시간 교통 및 버스위치, CCTV, 지하철, 버스노선, 거리뷰, 뮤지엄뷰 제공." property="twitter:description"/>
<meta content="summary_large_image" property="twitter:card"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<link href="/css/w_v1683023806668.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static/maps/m/icon/ic_ad_map1_96X96.png" rel=

In [78]:
soup.findall('li', {'class': '_item _lazyImgContainer'})

TypeError: 'NoneType' object is not callable

In [72]:
driver = load_driver()
driver.get(naver_nearby)
# wait = WebDriverWait(driver, 10)
# wait.until(EC.presence_of_element_located((By.XPATH, '//li[@class="_lazyImgContainer"]')))

/tmp/ipykernel_24323/1696479412.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [ ]:

    # explicitly wait until the page is loaded
    # driver.implicitly_wait(3)
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.place_section")))

    soup = BeautifulSoup(driver.page_source, 'html.parser')

In [46]:
get_place_header(34438090)

{'name': '배니닭강정',
 'type': '치킨,닭강정',
 'rating': '4.16/5',
 'n_visitor_reviews': '17,527',
 'n_blog_reviews': '1,682'}

In [44]:
driver.current_url

'https://m.map.naver.com/search2/interestSpot.naver?type=DINING&searchCoord=128.8906032;37.7499576'

In [29]:
import concurrent.futures

/tmp/ipykernel_24323/431188497.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  driver.find_elements_by_xpath('//li[@class="_item _lazyImgContainer"]')


[]

In [41]:
driver.page_source

'<html><head><title>404 Not Found</title></head>\n<body>\n<center><h1>404 Not Found</h1></center>\n<hr><center>nginx</center>\n\n\n\n\n\n\n\n\n</body></html>'

In [39]:
driver.get_attribute('outerHTML')

AttributeError: 'WebDriver' object has no attribute 'get_attribute'

In [30]:
max_results = 8
elements = driver.find_elements_by_css_selector('li._item._lazyImgContainer')[:max_results]



/tmp/ipykernel_24323/383044322.py:2: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  elements = driver.find_elements_by_css_selector('li._item._lazyImgContainer')[:max_results]


In [31]:
elements

[]

In [17]:
elements[0].get_attribute('data-id')

IndexError: list index out of range

In [ ]:

def get_place_details_(place_url):
    return get_place_details(place_url, query=query)

with concurrent.futures.ThreadPoolExecutor(len(elements)) as executor:
    results = list(executor.map(get_place_details_, [(f"https://m.place.naver.com/place/{element.get_attribute('data-id')}") for element in elements]))    

#results = []
for i in range(len(elements)):
    #place_url = f"https://m.place.naver.com/place/{element.get_attribute('data-id')}"
    #place_details = get_place_details(place_url, query)
    results[i]['longitude'] = float(elements[i].get_attribute('data-longitude'))
    results[i]['latitude'] = float(elements[i].get_attribute('data-latitude'))